In [ ]:
class QueryExpansion:
  def __init__(self, word_vectors, a1 , fasttext_vectors, weight_type = 'simple', dims=100):
    self.fasttext_vectors = fasttext_vectors
    self.weight_type = weight_type
    self.docs = a1
    self.dims = dims
    self.word_vectors = word_vectors
    self.vectorizer = TfidfVectorizer(use_idf = True, norm ='l1', ngram_range=(1,1), analyzer='word')
    self.vectorizer.fit_transform(self.docs)
    self.strings = []
    string = ''
    for item in self.vectorizer.get_feature_names():
      string += item
      string += ' '
      self.strings.append(item)
    self.vector = self.vectorizer.transform([string])
    self.vector = self.vector.toarray()
  
  def get_fasttext_average_tf_idf_wights(self,mesra):
    vector_sum = np.zeros(self.dims)
    words = mesra.split(' ')
    try:
      for word in words:
        index = self.strings.index(word)
        vector_sum += self.word_vectors[word] * self.vector[0][index]
    except Exception as e:
      pass
      # print(e.args[0])
    return vector_sum
  
  def get_fastext_average(self,mesra):
    counter = 0
    vector_sum = np.zeros(self.dims)
    words = mesra.split(' ')
    for word in words:
      try:
        vector_sum += self.word_vectors[word]
        counter += 1
      except:
        continue
      vector_sum /= counter
    return vector_sum

  def get_fasttext(self, mesra):
    if self.weight_type == 'tfidf':
      return self.get_fasttext_average_tf_idf_wights(mesra)
    elif self.weight_type == 'simple':
      return self.get_fastext_average(mesra)

  def query_expansion(self,query):
    number_of_related_queries = 100
    vector = self.get_fasttext(query)
    vector /= np.linalg.norm(vector)
    final_vector = np.matmul(self.fasttext_vectors, np.transpose(vector))
    sen_v2 = np.copy(self.docs)
    R = np.array(final_vector).argsort()[-number_of_related_queries:][::-1]
    NR = np.array(final_vector).argsort()[:number_of_related_queries][::-1]
    R_mean = np.mean(self.fasttext_vectors[R])
    NR_mean = np.mean(self.fasttext_vectors[NR])
    expanded_vector = vector + 1.5 * R_mean - 0.4 * NR_mean
    expanded_vector /= np.linalg.norm(expanded_vector)
    final_vector_expanded = np.matmul(self.fasttext_vectors, np.transpose(expanded_vector))
    R_expanded = np.array(final_vector).argsort()[-1:][::-1]
    expanded_query = sen_v2[R_expanded]
    return expanded_query



In [ ]:
# run this cell only once
word_vectors = KeyedVectors.load_word2vec_format('vec_file.vec')
with open('sen_v2.txt', 'r') as f:
  sen_v2 = f.read()
sen_v2 = sen_v2.split('\n')
sen_v2 = np.array(sen_v2)
fasttext_vectors = np.loadtxt('fasttext_vectors.txt')
nan_indexes = []
for i in range(len(final_vector)):
  if(np.isnan(fasttext_vectors[i]).any()):
    nan_indexes.append(i)
fasttext_vectors= np.delete(fasttext_vectors, nan_indexes, axis = 0)
sen_v2= np.delete(sen_v2, nan_indexes, axis = 0)
query_expander = QueryExpansion(word_vectors, sen_v2,  fasttext_vectors, 'simple' )

In [ ]:
# query expansion
query = 'کزین برتر اندیشه'
query_expander.query_expansion(query)